In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# WikipediaAPIWrapper:
This class allows you to search Wikipedia.
It limits the number of results (in this case, only 1 result) and controls how much of the article's text you get(up to 200 characters).

# WikipediaQueryRun:
This class runs the actual Wikipedia search using the WikipediaAPIWrapper.
When you ask it a question, it fetches relevant information from Wikipedia based on the limits you set.

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

db = FAISS.from_documents(documents,embeddings)
db_ret = db.as_retriever()
db_ret

USER_AGENT environment variable not set, consider setting it to identify your requests.
/var/folders/15/vc_j9b113v12tq235z56lnpr0000gn/T/ipykernel_34114/3662755599.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/Applications/anaconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x3140e6ec0>, search_kwargs={})

### Here we initialize the db and make the db as retrieve

In [5]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

### Same as Wikipedia

In [6]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = tool

In [7]:
arxiv.name

'arxiv'

In [8]:
from langchain.tools.retriever import create_retriever_tool
retrival_tool = create_retriever_tool(db_ret,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

### create_retriever_tool is more about creating a specialized tool for complex workflows, while db.as_retriever() is a simple way to 
### get a retriever from your vector database for immediate use.

### in simple english "create_retriever_tool" is used to make "db.as_retriever()" as a tool for other pipelines in langchain

In [9]:
tools = [wiki,arxiv,retrival_tool]

In [10]:
from langchain_ollama import OllamaLLM 
llm = OllamaLLM(model="llama3.2:latest")

In [11]:
from langchain import hub
# from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### This is a existing template

In [18]:
from langchain.agents import initialize_agent,AgentExecutor
from langchain.agents import create_tool_calling_agent

In [20]:
agent = create_tool_calling_agent(llm=llm,tools=tools,prompt=prompt)

ValueError: This function requires a .bind_tools method be implemented on the LLM.

In [14]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [15]:
print(agent.input_keys)

['', 'input', 'page_content']


In [16]:
Agent_exec = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [17]:
response = Agent_exec.invoke("What is LangSmith?")  # Just pass the string
print(response)

IndexError: list index out of range